In [1]:
!pip install flask jupyter_client


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install flask-cors


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import openai
print(openai.__version__)

1.28.1


In [17]:
pip install python-pptx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import jupyter_client
from flask import Flask, request, jsonify, render_template, send_file
from flask_cors import CORS
from pptx import Presentation
from pptx.util import Pt
import os
import io
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from openai import OpenAI
import base64
from pptx.enum.shapes import MSO_SHAPE_TYPE
import pandas as pd

In [41]:
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:3000"}})
# CORS(app)
donor_data=[]

In [42]:
@app.route('/send-gen-ppt', methods=['POST'])
def receive_message():
    # data = request.json  # Assuming JSON payload
    # if not data:
    #     return {"error": "No data received."}
    # ans = data['searchQuery']
    # print(ans)
    api_key = "sk-proj-OjvAkDmp5Viz6fMK98DnT3BlbkFJ2rrlkQUdax13LWhOOMA6"  # Fetch your API key from environment variables
    if not api_key:
        return {"error": "OpenAI API key not found."}
    with open("abcd.txt","r") as f:
        ans=f.read()

    print(api_key)
    client = OpenAI(api_key=api_key)

    TEMPLATE_FILE = r"D:\hackblr\frontend--search\pages\notebooks\template.pptx"
    if not os.path.exists(TEMPLATE_FILE):
        return {"error": "Template file not found."}

    presentation_title="ABCD"
    slide_contents = ans

    user_message_content = f"Create a presentation with 3 slides.\n" \
                           f"Presentation title: {presentation_title}\n" \
                           f"Slide content: {slide_contents}\n"

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "Generate a professional ppt for an ngo/npo looking to fundraise from donors. Assume yourself as a mid sized ngo/npo in Bengaluru, Karnataka named SmilesUp Foundation. Make a presentation for the NPO. For the PowerPoint presentation, please follow this structured format: Slide: Name of Title 1  Content: - 4 Bullet points about the topic in structured and detailed way Slide : Name of Title 2   Content: - 4 more bullet points without any repetition ,Slide: Name of Title 3  Content: 4 more bullet points which detailed review about the topic without repetition. I want All the bullet points to be given under '''Content''' keyword for every response and I need this structure and nothing else,remember you are pitching your idea and not giving suggestion to pitch"}
,
            {"role": "user", "content": user_message_content},
        ],
        model="gpt-3.5-turbo",
    )
    print(chat_completion.choices[0].message.content)
    if chat_completion.choices[0].message.content:
        generated_text = chat_completion.choices[0].message.content

        presentation = Presentation(TEMPLATE_FILE)

        slides = generated_text.split("Slide")

        if len(slides) - 1 <= len(presentation.slides):
            for i, slide_content in enumerate(slides[1:], start=0):
                content_lines = slide_content.strip().split("\n")
                if len(content_lines) >= 2:
                    slide_title = content_lines[0].strip()
                    slide_content = content_lines[1:]  # Extract bullet points

                    slide = presentation.slides[i]
                    title = slide.shapes.title
                    content_box = slide.placeholders[1]

                    title.text = slide_title
                    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)

                    content_box.text = ""  # Clear existing text
                    content_frame = content_box.text_frame
                    content_frame.clear()  # Clear existing paragraphs
                    for point in slide_content:
                        p = content_frame.add_paragraph()
                        p.text = point.strip()
                        p.font.color.rgb = RGBColor(255, 255, 255)
                        p.space_after = Pt(14)  # Adjust spacing between bullet points

        else:
            return {"error": "The template has fewer slides than required."}


        pptx_bytes = io.BytesIO()
        presentation.save(pptx_bytes)
        pptx_bytes.seek(0)

        # Convert the PowerPoint file data to a base64-encoded string
        pptx_file_base64 = base64.b64encode(pptx_bytes.getvalue()).decode('utf-8')

        a=chat_completion.choices[0].message.content.replace("Slide:","Title:")
        a=a.replace("-","\n")
        # Create a custom response object containing both the generated text and the base64-encoded PowerPoint file
        response_data = {
            "generated_text": a,
            "pptx_file_base64": pptx_file_base64
        }

        return jsonify(response_data)

    else:
        return {"error": "No response from OpenAI."}


@app.route("/receive-donor-name", methods=['POST'])
def return_home():
    data = request.json  # Assuming JSON payload
    if not data:
        return {"error": "No data received."}
    ans = data['searchQuery']

    return jsonify({
        "message" : "Message-Gone"
    })
app.run(debug=True, use_reloader=False, port=8000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [12/May/2024 16:45:44] "OPTIONS /send-gen-ppt HTTP/1.1" 200 -


sk-proj-OjvAkDmp5Viz6fMK98DnT3BlbkFJ2rrlkQUdax13LWhOOMA6


127.0.0.1 - - [12/May/2024 16:45:47] "POST /send-gen-ppt HTTP/1.1" 200 -


Slide: Education 
Content:
- Providing quality education to underprivileged children
- Running innovative educational programs to enhance learning outcomes
- Partnering with schools and institutions to improve educational infrastructure
- Conducting skill development workshops for students

Slide: Empowerment
Content:
- Empowering women through vocational training and entrepreneurship opportunities
- Providing healthcare services to improve the well-being of communities
- Advocating for gender equality and women’s rights
- Supporting sustainable livelihood initiatives for marginalized groups

Slide: Impact
Content:
- Highlighting success stories of individuals whose lives have been positively impacted
- Showcasing the measurable outcomes of our interventions in the community
- Sharing testimonials from beneficiaries, partners, and stakeholders
- Illustrating our commitment to transparency, accountability, and sustainability


127.0.0.1 - - [12/May/2024 16:54:26] "OPTIONS /send-gen-ppt HTTP/1.1" 200 -


sk-proj-OjvAkDmp5Viz6fMK98DnT3BlbkFJ2rrlkQUdax13LWhOOMA6


127.0.0.1 - - [12/May/2024 16:54:30] "POST /send-gen-ppt HTTP/1.1" 200 -


Slide: Introduction
Content:
- SmilesUp Foundation: Bengaluru based NGO committed to animal welfare
- Established track record of impactful projects in Karnataka
- Dedicated team of professionals and volunteers
- Focus on holistic animal care and community education

Slide: Funding Needs
Content:
- Vaccination and medical care for stray animals
- Rescue and rehabilitation programs
- Awareness campaigns on animal welfare
- Support for sustainable projects for long-term impact

Slide: Donor Opportunities
Content:
- Tax-exempt donations for both individuals and corporations
- Transparent allocation of funds with regular updates
- Sponsorship opportunities for specific projects
- Collaboration and recognition opportunities for donors


In [ ]:
import pandas as pd

def find_data_for_name(name, csv_path):
    # Read the dataset
    df = pd.read_csv(csv_path)
    
    # Check if the name is present in the second column
    filtered_df = df[df.iloc[:, 1] == name]
    
    # If the name is found, return corresponding data from columns 4, 5, and 6
    if not filtered_df.empty:
        data = filtered_df.iloc[:, [3, 4, 5]].values.tolist()
        return data
    else:
        return "Name not found in the dataset"

# Example usage
ans = "Mr.M.K.Sehgal "
dataset_path = r'C:\Users\adith\Desktop\frontend--search\pages\notebooks\datasets\donor_history.csv'

result = find_data_for_name(ans, dataset_path)
print(result)


[['Provided $35,000 in funding for a NeuroScience lecture series at a local library.', 'Provided $35,000 in funding for a NeuroScience lecture series at a local library.', 'Provided $35,000 in funding for a NeuroScience lecture series at a local library.']]


In [ ]:
import pandas as pd

def find_data_for_name(name, csv_path):
    # Read the dataset
    df = pd.read_csv(csv_path)
    
    # Convert the name to string
    name = str(name)
    
    # Check if the name is present in the second column
    filtered_df = df[df.iloc[:, 1].astype(str).str.strip() == name]
    
    # If the name is found, return corresponding data from columns 4, 5, and 6
    if not filtered_df.empty:
        # Fill NaN values with empty strings
        filtered_df.fillna('', inplace=True)
        data = filtered_df.iloc[:, [3, 4, 5]].values.tolist()
        return data
    else:
        return "Name not found in the dataset"

# Example usage
ans = "Mr.M.K.Sehgal "
dataset_path = r'C:\Users\adith\Desktop\frontend--search\pages\notebooks\datasets\donor_history.csv'

result = find_data_for_name(ans, dataset_path)
print(result)


Name not found in the dataset


In [ ]:
df.head()

,Cause,Donor_Name,Amount,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NeuroScience,DrR.P. & Mrs.T.Sengupta,$1 Million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NeuroScience,"Armenian Church of Nazareth, Calcutta","$5,00,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NeuroScience,Sunita Dalmia Foundation,"$2,50,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NeuroScience,Madhu Saraf,"$2,50,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NeuroScience,"Infinity IT Park, Kolkata","$2,50,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
